In [115]:
# Importare le librerie
import requests
from bs4 import BeautifulSoup as bs
import csv
import datetime
import locale

urls =[{
    "link": "https://www.imdb.com/title/tt5180504/episodes/",
    "csv" : "THEWITCHER.csv"
    },{
    "link": "https://www.imdb.com/title/tt3581920/episodes/",
    "csv" : "THELASTOFUS.csv"
}]

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
    "accept-language": "it-IT,it;q=0.9,en-US;q=0.8,en;q=0.7" 
}
image_class = "srcset"


def get_data(soup_episode):
    data_tag = soup_episode.find("span",class_="sc-f2169d65-10 iZXnmI")
    if data_tag != None:
        return formatta_data (data_tag.text.strip())
    return None

def formatta_data(data_in):
    data_splitted = data_in.split(",")[-1] 
    locale.setlocale(locale.LC_TIME, "it_IT")
    data = datetime.datetime.strptime(data_splitted, " %d %b %Y")
    data_out = data.strftime("%d/%m/%Y")
    return data_out

def get_tot_value(soup_episode):
    tot_value_tag = soup_episode.find("span",class_="ipc-rating-star--voteCount")
    if tot_value_tag != None:
        value=tot_value_tag.text.strip()
        value_clean=value.replace("(","").replace(")","").replace(".","")
        return int(value_clean)
    return None
def get_rank(soup_episode):
    rank_tag = soup_episode.find("span",class_="ipc-rating-star")
    if rank_tag != None:
        rank=rank_tag.text.strip()
        rank_clean=rank.split("/")
        if len(rank_clean) > 1:
            rank_float=rank_clean[0].replace(",",".")
            return  float(rank_float)
        else:
            rank_float=rank.replace(",",".")
            return  float(rank_float)
    return None
    
def get_identifier_title(soup_episode):
    title_tag =soup_episode.find("div",class_="ipc-title__text")
    if title_tag != None:
        title = title_tag.text.strip()
        title_splitted = title.split(" ∙ ")
        if len (title_splitted)>1:
            return title_splitted[0],title_splitted[1]
        else:
            return title,None
    return None,None
                                 
def get_last_link(input_string):
    link_list = input_string.split(", ")
    last_link = link_list[-1]
    last_link_split = last_link.split(" ")
    return last_link_split [0]

def getImage_episode(soup_episode):
    image_tag = soup_episode.find("img")
    if image_tag and image_class in image_tag.attrs:
        image_list = image_tag[image_class]
        return get_last_link(image_list)
    
def dictepisode(image,identifier,title,tot_value,data,rank):
    episodes_info ={
        "Image":image,
        "Identifier":identifier,
        "Title":title,
        "Tot_value":tot_value,
        "Data":data,
        "Rank":rank
    }
    return episodes_info

def into_csv(array, nome_file):
    with open(nome_file, "w", newline="") as file:
        writer = csv.DictWriter(file, fieldnames=array[0].keys())
        writer.writeheader()
        for dizionario in array:
            writer.writerow(dizionario)
            
for url in urls :
    episodes=[]
    response = requests.get(url["link"], headers=headers)
    if response.status_code ==200:
        soup = bs(response.content,"html.parser")
        cointainer = soup.find("section",class_="sc-7b9ed960-0 jNjsLo")
        episode_list = cointainer.find_all("article",class_="sc-282bae8e-1 dSEzwa episode-item-wrapper")
        for episode in episode_list:
            image = getImage_episode(episode)
            identifier,title = get_identifier_title(episode)
            tot_value = get_tot_value(episode)
            data = get_data(episode)
            rank = get_rank(episode)
            dictionary= dictepisode(image,identifier,title,tot_value,data,rank)
            episodes.append(dictionary)
    
        into_csv(episodes,url["csv"])
        print("salvo il file in csv "+ url["csv"])
    else:
        print (response)
        print("non sono riuscito a recuperare la pagina")


salvo il file in csv THEWITCHER.csv
salvo il file in csv THELASTOFUS.csv
